In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!mkdir /content/drive/MyDrive/lsml2
%cd /content/drive/MyDrive/lsml2/

/content/drive/MyDrive/lsml2


In [ ]:
!pip install simplejson

     |████████████████████████████████| 130 kB 14.4 MB/s 


In [ ]:
import gzip
import simplejson

def parse(filename):
  f = gzip.open(filename, 'rt', encoding = 'latin 1')
  entry = {}
  for l in f:
    l = l.strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry

In [ ]:
raw = []
for e in parse('/content/drive/MyDrive/lsml2/sentiment_analysis/docker_data/finefoods.txt.gz'):
  raw.append(e)

In [ ]:
import pandas as pd
df = pd.DataFrame(raw)

In [ ]:
df.describe()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
count,568454,568454,568454,568454,568454,568454,568454,568454
unique,74258,256059,218418,1571,5,3168,295744,393579
top,B007JFMH8M,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",0/0,5.0,1350345600,Delicious!,"This review will make me sound really stupid, ..."
freq,913,448,451,270052,363122,1143,2462,199


In [ ]:
df

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...
568457,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0/0,2.0,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568458,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2/2,5.0,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568459,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1/1,5.0,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...
568460,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0/0,5.0,1338422400,Great Honey,"I am very satisfied ,product is as advertised,..."


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 75.3 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 76.0 MB/s 
     |████████████████████████████████| 6.5 MB 64.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel

import torch

class BERTDataset:
    def __init__(self, review, target):
        self.review = review
        self.target = target
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case =True)
        self.max_len = 512

    def __len__(self):
        return len(self.review)

    def __getitem__(self, item):
        review = str(self.review[item])
        review = " ".join(review.split())

        inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        padding_length = self.max_len - len(ids)
        ids = ids +([0]*padding_length)
        mask = mask + ([0]*padding_length)
        token_type_ids = token_type_ids + ([0]* padding_length)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float),
        }

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device, dtype = torch.long)
        token_type_ids = token_type_ids.to(device, dtype = torch.long)
        mask = mask.to(device, dtype = torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            ids = ids,
            mask = mask,
            token_type_ids = token_type_ids
        )

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(
                ids=ids, 
                mask=mask, 
                token_type_ids=token_type_ids
                )

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    
    return fin_outputs, fin_targets   

In [ ]:
import transformers
import torch.nn as nn

class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased", return_dict=False)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)

    def forward(self, ids, mask, token_type_ids):
        output_1, output_2 = self.bert(
            ids,
            attention_mask = mask,
            token_type_ids = token_type_ids
        )

        bert_output = self.bert_drop(output_2)
        output = self.out(bert_output)
        return output

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup


dfx = pd.DataFrame(df, columns = ['review/score','review/text'])[:568460]
dfx.rename(columns={'review/text':'review','review/score':'sentiment'}, inplace=True)

In [ ]:
dfx

,sentiment,review
0,5.0,I have bought several of the Vitality canned d...
1,1.0,Product arrived labeled as Jumbo Salted Peanut...
2,4.0,This is a confection that has been around a fe...
3,2.0,If you are looking for the secret ingredient i...
4,5.0,Great taffy at a great price. There was a wid...
...,...,...
568455,5.0,My only complaint is that there's so much of i...
568456,5.0,Great for sesame chicken..this is a good if no...
568457,2.0,I'm disappointed with the flavor. The chocolat...
568458,5.0,"These stars are small, so you can give 10-15 o..."


In [ ]:
dfx['sentiment'] = dfx.sentiment.astype(float)

In [ ]:
dfx['sentiment'].describe()

count    568453.000000
mean          4.183197
std           1.310437
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: sentiment, dtype: float64

In [ ]:
dfx.sentiment = dfx.sentiment.apply(
    lambda x: 1 if x > 3.0  else 0
)

In [ ]:
dfx['sentiment']

0         1
1         0
2         1
3         0
4         1
         ..
568455    1
568456    1
568457    0
568458    1
568459    1
Name: sentiment, Length: 568460, dtype: int64

In [ ]:
df_train, df_valid = model_selection.train_test_split(
    dfx,
    test_size = 0.1,
    random_state = 7,
    stratify = dfx.sentiment.values
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)


train_dataset = BERTDataset(
    review = df_train.review.values,
    target = df_train.sentiment.values
)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    num_workers = 2
)

valid_dataset = BERTDataset(
    review = df_valid.review.values,
    target = df_valid.sentiment.values
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=4,
    num_workers = 1
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTBaseUncased()
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

num_train_steps = int(len(df_train) / 8 * 10)
optimizer = AdamW(optimizer_parameters, lr=1e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_train_steps
)

best_accuracy = 0
for epoch in range(1):
    train_fn(train_data_loader, model, optimizer, device, scheduler)
    outputs, targets = eval_fn(valid_data_loader, model, device)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    print(f"Accuracy Score = {accuracy}")
    if accuracy > best_accuracy:
        torch.save(model.state_dict(), "my_model.bin")
        best_accuracy = accuracy

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Accuracy Score = 0.9602082820251205


In [ ]:
accuracy

0.9602082820251205

In [ ]:
best_accuracy

0.9602082820251205

In [ ]:
import transformers

DEVICE = 'cpu'
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
BERT_PATH = "/root/docker_data/bert_base_uncased"
MODEL_PATH = "/root/docker_data/model.bin"
TRAINING_FILE = "/root/docker_data/IMDB_dataset.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    BERT_PATH,
    do_lower_case =True
)

In [ ]:
import torch
import flask
import time

import functools
import torch.nn as nn

DEVICE = 'cpu'
PREDICTION_DICT = dict()


def sentence_prediction(sentence):
    tokenizer = TOKENIZER = transformers.BertTokenizer.from_pretrained(
    '/content/drive/MyDrive/lsml2/bert_base_uncased',
    do_lower_case =True
    )
    max_len = 512
    review = str(sentence)
    review = " ".join(review.split())
    
    inputs = tokenizer.encode_plus(
        review, None, add_special_tokens=True, max_length=max_len
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

    ids = ids.to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)

    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)

    outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    #return outputs[0][0]
    return outputs[0][0]

In [ ]:
DEVICE = 'cpu'
MODEL = BERTBaseUncased()
MODEL.load_state_dict(torch.load('/content/drive/MyDrive/lsml2/my_model.bin' , map_location=torch.device('cpu')))
MODEL.to(DEVICE)
MODEL.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
sentence_prediction('worst scenario I\'ve ever seen')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0.00219039